In [3]:
from riotwatcher import LolWatcher, ApiError
import time
import pandas as pd

In [4]:
# === Set up ===
api_key = 'RGAPI-db3b2160-c70a-4435-b7e4-4e5bf0fd2aa6'  # Replace with your API key
watcher = LolWatcher(api_key)
region = 'na1'  # Use appropriate region (e.g., 'euw1', 'kr', 'na1')

# Seed summoner names (can be real or your own list)
summoners = ['X8Scorpio8X']

match_ids_collected = set()
matches_data = []

def safe_api_call(func, *args, retries=3, **kwargs):
    for _ in range(retries):
        try:
            return func(*args, **kwargs)
        except ApiError as err:
            if err.response.status_code == 429:
                print("Rate limit hit. Sleeping...")
                time.sleep(120)
            elif err.response.status_code == 403:
                raise Exception("Invalid or expired API key.")
            else:
                print(f"API error {err.response.status_code}. Skipping.")
                return None

def get_team_features(match):
    features = []
    for team in match['info']['teams']:
        stats = {
            'win': team['win'],
            'baron_kills': team['objectives']['baron']['kills'],
            'dragon_kills': team['objectives']['dragon']['kills'],
            'tower_kills': team['objectives']['tower']['kills'],
            'inhibitor_kills': team['objectives']['inhibitor']['kills'],
            'rift_herald_kills': team['objectives']['riftHerald']['kills'],
        }
        features.append(stats)
    return features

# === Main loop ===
for name in summoners:
    try:
        summoner = safe_api_call(watcher.summoner.by_account, region, name)
        puuid = summoner['puuid']
        match_ids = safe_api_call(watcher.match.matchlist_by_puuid, 'americas', puuid, type='ranked', count=10)
        if match_ids:
            for match_id in match_ids:
                if match_id in match_ids_collected:
                    continue
                match_data = safe_api_call(watcher.match.by_id, 'americas', match_id)
                if match_data:
                    team_stats = get_team_features(match_data)
                    matches_data.extend(team_stats)
                    match_ids_collected.add(match_id)
                if len(matches_data) >= 100:
                    break
        if len(matches_data) >= 100:
            break
    except Exception as e:
        print(f"Error with summoner {name}: {e}")
        continue

# === Save to DataFrame ===
df = pd.DataFrame(matches_data)
print(df.head())


API error 400. Skipping.
Error with summoner X8Scorpio8X: 'NoneType' object is not subscriptable
Empty DataFrame
Columns: []
Index: []


In [8]:
api_key = 'RGAPI-db3b2160-c70a-4435-b7e4-4e5bf0fd2aa6'
watcher = LolWatcher(api_key)
region = 'na1'

summoner = watcher.summoner.by_name(region, 'X8Scorpio8X')
print(summoner)

HTTPError: 400 Client Error: Bad Request for url: https://na.api.riotgames.com/lol/summoner/v4/summoners/by-name/X8Scorpio8X